In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import yaml
import os

In [4]:
import pandas as pd

In [5]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans

In [6]:
from utils import plot_paired_imgs
from utils import plot_sample_imgs
from utils import read_images

from model.cae import CAE

Using TensorFlow backend.


In [7]:
from keras.preprocessing.image import ImageDataGenerator

## Read data

In [8]:
with open('conf/user_conf.yaml', 'r') as f:
    conf = yaml.load(f)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [9]:
patches_path = os.path.join(conf['data_path'], 'slides', 'patches')
image_size = conf['wsi']['patch_size']

In [11]:
train_df = pd.read_csv(os.path.join(conf['data_path'], 'train.csv'), sep='|')
test_df = pd.read_csv(os.path.join(conf['data_path'], 'test.csv'), sep='|')

In [13]:
test_df.head(3)

,filename,slide_id,case_id,sample_id,disease_type,sample_type,percent_normal_cells,percent_stromal_cells,percent_tumor_cells,percent_tumor_nuclei
0,TCGA-HZ-8317-01A-01-TS1_42_07.png,TCGA-HZ-8317-01A-01-TS1,TCGA-HZ-8317,TCGA-HZ-8317-01A,Ductal and Lobular Neoplasms,Primary Tumor,26.0,42.0,32.0,10.0
1,TCGA-FZ-5920-01A-01-BS1_047_086.png,TCGA-FZ-5920-01A-01-BS1,TCGA-FZ-5920,TCGA-FZ-5920-01A,Ductal and Lobular Neoplasms,Primary Tumor,5.0,40.0,55.0,75.0
2,TCGA-F2-6879-01A-01-TS1_35_47.png,TCGA-F2-6879-01A-01-TS1,TCGA-F2-6879,TCGA-F2-6879-01A,Adenomas and Adenocarcinomas,Primary Tumor,NaN,40.0,55.0,80.0


In [14]:
imgen = ImageDataGenerator(rescale=1/255)

train_generator = imgen.flow_from_dataframe(train_df, directory=patches_path, x_col='filename', 
                                            target_size=(image_size, image_size), color_mode='rgb', class_mode=None, shuffle=False)

test_generator = imgen.flow_from_dataframe(test_df, directory=patches_path, x_col='filename', 
                                           target_size=(image_size, image_size), color_mode='rgb', class_mode=None, shuffle=False)

Found 58948 images.
Found 13317 images.


## CAE Predictions 

__Load CAE__

In [15]:
model = 'model_test'

In [16]:
os.path.join(conf['data_path'], 'modelling', 'models', model)

'/home/jupyter/data/modelling/models/model_test'

In [18]:
cae = CAE(path=os.path.join(conf['data_path'], 'models', model), load=True)

Instructions for updating:
Colocations handled automatically by placer.


/home/jupyter/TFM/model/cae.py:80: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  params = yaml.load(f)


In [ ]:
test_enc = cae.encoder.p